# Import

In [2]:
import pandas as pd
import numpy as np
import sqlite3
import matplotlib.pyplot as plt
import seaborn as sns
import re
import plotly.express as px
from jupyterthemes import jtplot
jtplot.style()

# Read in

In [18]:
gach = pd.read_csv('../data/neo4j/full_refs.csv')
gach_coms = pd.read_csv('../data/neo4j/gach_simple_communities.csv')

In [20]:
gach_coms['organization'].unique()

array([nan, 'NASHVILLE GENERAL HOSPITAL',
       'HCA HEALTH SERVICES OF TENNESSEE, INC.',
       'VANDERBILT UNIVERSITY MEDICAL CENTER', 'SETON CORPORATION',
       'MAURY REGIONAL HOSPITAL', 'NORTHCREST MEDICAL CENTER',
       'SAINT THOMAS STONES RIVER HOSPITAL, LLC',
       'SAINT THOMAS WEST HOSPITAL', 'RIVER PARK HOSPITAL LLC',
       'SAINT THOMAS RIVER PARK HOSPITAL, LLC',
       'TRUSTPOINT HOSPITAL, LLC', 'SAINT THOMAS RUTHERFORD HOSPITAL',
       'SHELBYVILLE HOSPITAL COMPANY LLC',
       'SAINT THOMAS DEKALB HOSPITAL, LLC', 'SRHS HOLDINGS LLC',
       'SMITH COUNTY MEMORIAL HOSPITAL, LLC',
       'RIVERVIEW MEDICAL CENTER LLC',
       'SMITH COUNTY MEMORIAL HOSPITAL LLC',
       'CENTRAL TENNESSEE HOSPITAL CORPORATION',
       'SUMNER REGIONAL HEALTH SYSTEMS, INC',
       'SUMNER REGIONAL MEDICAL CENTER LLC',
       'WILLIAMSON COUNTY HOSPITAL DISTRICT',
       'TROUSDALE MEDICAL CENTER LLC',
       'HENDERSONVILLE HOSPITAL CORPORATION',
       'MACON COUNTY GENERAL HOSPITA

In [28]:
gach_coms_counts = gach_coms.groupby('communityId')['npi'].count().reset_index().rename(columns = {'npi':'count'})

In [32]:
real_coms = gach_coms_counts.loc[gach_coms_counts['count'] > 0, 'communityId']
gach_coms = gach_coms[gach_coms['communityId'].isin(real_coms)]

In [35]:
prov_coms = gach_coms[gach_coms['organization'].isnull()]
hosp_coms = gach_coms[~gach_coms['organization'].isnull()]

In [39]:
gach_coms_counts = gach_coms.groupby('communityId')['npi'].count().reset_index().rename(columns = {'npi':'count'})
hosp_coms_counts = gach_coms_counts.merge(hosp_coms).sort_values('count', ascending = False)

In [45]:
hosp_addresses = gach[['to_npi', 'address_hospital']].drop_duplicates()

In [46]:
hosp_coms_counts.merge(hosp_addresses, left_on = 'npi', right_on = 'to_npi')

,communityId,count,npi,name,organization,to_npi,address_hospital
0,4058,1885,1.396882e+09,NaN,VANDERBILT UNIVERSITY MEDICAL CENTER,1396882205,"1211 MEDICAL CENTER DRIVE, NASHVILLE, TN 37232"
1,4058,1885,1.558409e+09,NaN,VANDERBILT UNIVERSITY MEDICAL CENTER,1558408633,"1601 23RD AVE S, NASHVILLE, TN 37212"
2,5464,819,1.629026e+09,NaN,SAINT THOMAS WEST HOSPITAL,1629025648,"4220 HARDING RD, NASHVILLE, TN 37205"
3,5464,819,1.518346e+09,NaN,"SAINT THOMAS RIVER PARK HOSPITAL, LLC",1518346469,"1559 SPARTA ST, MCMINNVILLE, TN 37110"
4,5464,819,1.780779e+09,NaN,SAINT THOMAS WEST HOSPITAL,1780778969,"2000 CHURCH ST, NASHVILLE, TN 37236"
5,5464,819,1.992861e+09,NaN,SETON CORPORATION,1992861314,"2000 CHURCH ST, NASHVILLE, TN 37236"
6,2931,700,1.932146e+09,NaN,"HCA HEALTH SERVICES OF TENNESSEE, INC.",1932146032,"2300 PATTERSON ST, NASHVILLE, TN 37203"
7,2931,700,1.720032e+09,NaN,"HCA HEALTH SERVICES OF TENNESSEE, INC.",1720032345,"391 WALLACE RD, NASHVILLE, TN 37211"
8,2931,700,1.265487e+09,NaN,"HCA HEALTH SERVICES OF TENNESSEE, INC.",1265487193,"313 N MAIN ST, ASHLAND CITY, TN 37015"
9,2931,700,1.992776e+09,NaN,"HCA HEALTH SERVICES OF TENNESSEE, INC.",1992776405,"200 STONECREST BLVD, SMYRNA, TN 37167"
